In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time 

In [2]:
print(tf.__version__)

2.9.1


In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128 
IMAGE_SIZE = [180, 180]
EPOCHS = 25

In [4]:
filenames = tf.io.gfile.glob(str('train/*/*'))
filenames.extend(tf.io.gfile.glob(str('val/*/*')))

train_filenames, val_filenames = train_test_split(filenames, test_size=0.2)

In [5]:
# import pickle

# pickle.dump(train_filenames, open('train_filenames.pkl', 'wb'))
# pickle.dump(val_filenames, open('val_filenames.pkl', 'wb'))

In [6]:
len(train_filenames), len(val_filenames)

(4185, 1047)

In [7]:
COUNT_NORMAL = len([filename for filename in train_filenames if "NORMAL" in filename])
print("Normal images count in training set: " + str(COUNT_NORMAL))

COUNT_PNEUMONIA = len([filename for filename in train_filenames if "PNEUMONIA" in filename])
print("Pneumonia images count in training set: " + str(COUNT_PNEUMONIA))

Normal images count in training set: 1100
Pneumonia images count in training set: 3085


In [8]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)

for f in train_list_ds.take(5):
    print(f.numpy())

b'train/PNEUMONIA/person58_bacteria_273.jpeg'
b'train/PNEUMONIA/person853_bacteria_2775.jpeg'
b'train/PNEUMONIA/person547_bacteria_2292.jpeg'
b'train/PNEUMONIA/person1411_bacteria_3607.jpeg'
b'val/PNEUMONIA/person1946_bacteria_4874.jpeg'


2022-07-29 17:28:41.948794: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-29 17:28:41.948839: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-7-97): /proc/driver/nvidia/version does not exist
2022-07-29 17:28:41.952609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
print("Training images count: " + str(TRAIN_IMG_COUNT))

VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
print("Validating images count: " + str(VAL_IMG_COUNT))

Training images count: 4185
Validating images count: 1047


In [10]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    return parts[-2] == "PNEUMONIA"

In [11]:
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, IMAGE_SIZE)

In [12]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [13]:
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [14]:
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (180, 180, 3)
Label:  True


In [15]:
test_list_ds = tf.data.Dataset.list_files(str('test/*/*'))
TEST_IMAGE_COUNT = tf.data.experimental.cardinality(test_list_ds).numpy()
test_ds = test_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE)

TEST_IMAGE_COUNT

624

In [16]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [17]:
train_ds = prepare_for_training(train_ds)
val_ds = prepare_for_training(val_ds)

image_batch, label_batch = next(iter(train_ds))

2022-07-29 17:28:46.170604: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [18]:
def conv_block(filters):
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    )
    
    return block

In [19]:
def dense_block(units, dropout_rate):
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [20]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
        
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(16, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        conv_block(32),
        conv_block(64),
        
        conv_block(128),
        tf.keras.layers.Dropout(0.2),
        
        conv_block(256),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        dense_block(512, 0.7),
        dense_block(128, 0.5),
        dense_block(64, 0.3),
        
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    return model

In [21]:
initial_bias = np.log([COUNT_PNEUMONIA/COUNT_NORMAL])
initial_bias

array([1.03124148])

In [22]:
weight_for_0 = (1 / COUNT_NORMAL)*(TRAIN_IMG_COUNT)/2.0 
weight_for_1 = (1 / COUNT_PNEUMONIA)*(TRAIN_IMG_COUNT)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 1.90
Weight for class 1: 0.68


In [23]:
model = build_model()

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=METRICS
)

In [24]:
start = time.time()
history = model.fit(
    train_ds,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_ds,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
)
end = time.time() - start
print("total training time: ", end)

Epoch 1/25
32/32 [==============================] - 77s 2s/step - loss: 0.5326 - accuracy: 0.7358 - precision: 0.9344 - recall: 0.6897 - val_loss: 0.6174 - val_accuracy: 0.7607 - val_precision: 0.7607 - val_recall: 1.0000
Epoch 2/25
32/32 [==============================] - 60s 2s/step - loss: 0.3109 - accuracy: 0.8833 - precision: 0.9742 - recall: 0.8644 - val_loss: 0.5452 - val_accuracy: 0.7656 - val_precision: 0.7656 - val_recall: 1.0000
Epoch 3/25
32/32 [==============================] - 58s 2s/step - loss: 0.2435 - accuracy: 0.9124 - precision: 0.9778 - recall: 0.9019 - val_loss: 0.5915 - val_accuracy: 0.7598 - val_precision: 0.7598 - val_recall: 1.0000
Epoch 4/25
32/32 [==============================] - 57s 2s/step - loss: 0.2059 - accuracy: 0.9290 - precision: 0.9817 - recall: 0.9210 - val_loss: 0.6621 - val_accuracy: 0.7646 - val_precision: 0.7646 - val_recall: 1.0000
Epoch 5/25
32/32 [==============================] - 58s 2s/step - loss: 0.1637 - accuracy: 0.9382 - precision: 0

In [25]:
print('Training time (mins): {}'.format(end/60))

Training time (mins): 24.53151824871699


In [26]:
start_eval = time.time()

loss, acc, prec, rec = model.evaluate(test_ds)

end_eval = time.time() - start_eval

print("loss: ", loss)

print("acc: ", acc)

print("prec: ", prec)

print("rec: ", rec)

print("total prediction time: ", end_eval)

5/5 [==============================] - 4s 591ms/step - loss: 2.0823 - accuracy: 0.6410 - precision: 0.6352 - recall: 1.0000
loss:  2.0822575092315674
acc:  0.6410256624221802
prec:  0.6351791620254517
rec:  1.0
total prediction time:  3.8138153553009033
